# Agents in Team

### A single Agent can only say create a short story for us.
### but with a team whre many agents work together towards a common goal they can help us in writing or even helping to review, edit etc.

In [2]:
# Import asyncio for running asynchronous functions
import asyncio

# Import OpenAIChatCompletionClient from autogen_ext to interact with OpenAI models
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Import dotenv to load environment variables from a .env file
from dotenv import load_dotenv

# Import os to access environment variables
import os

# Load environment variables from a .env file into the runtime environment
load_dotenv()

# Retrieve the OpenAI API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')

# Create an OpenAI chat model client using the 'gpt-4o' model
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

# Create another OpenAI chat model client using the 'gpt-4' model
# This allows for task separation or comparison between different models
model_client_2 = OpenAIChatCompletionClient(model='gpt-4', api_key=api_key)


In [3]:
# Importing the AssistantAgent class from the autogen_agentchat.agents module
from autogen_agentchat.agents import AssistantAgent

# Creating an assistant agent to solve complex Data Structures and Algorithms (DSA) problems
dsa_solver = AssistantAgent(
    name='Complex_DSA_Solver',  # Unique name for the agent
    model_client=model_client,  # Language model client used for generating responses
    description='A DSA solver',  # Short description of the agent's role
    system_message="You give code in python to solve complex DSA problems. Give under 100 words"  # Instruction guiding the agent's behavior
)

# Creating an assistant agent to review code generated by the DSA solver
code_reviewer = AssistantAgent(
    name='CODE_REVEIWER',  # Unique name for the reviewer agent
    model_client=model_client_2,  # Different model client, possibly optimized for reviewing
    description='A Code Reviewer',  # Description of the agent
    system_message="You review the code given by the complex_dsa_solver and make sure it is optimized. Give under 10 words"  # Reviewer-specific behavior instruction
)

# Creating an assistant agent to edit the code for readability and add comments
code_editor = AssistantAgent(
    name='CODE_EDITOR',  # Unique name for the editor agent
    model_client=model_client,  # Same model client as the DSA solver
    description='A Code editor',  # Description of the agent
    system_message="You make the code easy to understand and add comments wherever required. Give under 10 words"  # Instruction for editing and enhancing clarity
)


In [4]:
# Importing RoundRobinGroupChat to enable a round-robin style multi-agent conversation
from autogen_agentchat.teams import RoundRobinGroupChat

# Importing TextMessage to create user prompts/messages for the agent team
from autogen_agentchat.messages import TextMessage

# Initializing a round-robin group chat with 3 participants (agents):
# 1. dsa_solver: solves DSA problems
# 2. code_reviewer: reviews the generated code
# 3. code_editor: edits the code to make it readable and adds comments
team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    max_turns=15  # Maximum number of message-passing turns before the conversation stops
)

# Defining an asynchronous test function to run the group chat
async def test_team():
    # Creating a task message from the user, asking for a simple Hello World code
    task = TextMessage(content='Write a simple Hello world code ?', source='User')
    
    # Running the round-robin interaction with the initial task
    result = await team.run(task=task)

    # Iterating over each message in the result and printing it
    for each_agent_message in result.messages:
        print(f'{each_agent_message}')
        print('\n\n')  # Adding space between messages for better readability

# Run the async test function
await test_team()


id='ae38ab8e-8589-4cc0-9b8b-2b0f95ecacb0' source='User' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 15, 19, 45, 50, 974762, tzinfo=datetime.timezone.utc) content='Write a simple Hello world code ?' type='TextMessage'



id='48acc80c-602a-4903-a7ec-8ec81d23f980' source='Complex_DSA_Solver' models_usage=RequestUsage(prompt_tokens=37, completion_tokens=46) metadata={} created_at=datetime.datetime(2025, 7, 15, 19, 45, 52, 873449, tzinfo=datetime.timezone.utc) content='Certainly! Here\'s a simple "Hello, World!" program in Python:\n\n```python\nprint("Hello, World!")\n```\n\nThis code uses the `print()` function to output the string "Hello, World!" to the console.' type='TextMessage'



id='147b2329-7844-4a5a-b515-bd12f0693c1f' source='CODE_REVEIWER' models_usage=RequestUsage(prompt_tokens=100, completion_tokens=6) metadata={} created_at=datetime.datetime(2025, 7, 15, 19, 45, 54, 428207, tzinfo=datetime.timezone.utc) content='Code is simple and optimized.' type='Text

In [5]:
# Resetting the team conversation state
await team.reset()  # Clear previous conversation history so the team can start fresh.

# Import TaskResult to check if a message is the final result (not just an agent's message)
from autogen_agentchat.base import TaskResult

# Run the team in streaming mode using `run_stream` with an input task (string form)
# This allows you to handle messages as they are generated in real-time
async for message in team.run_stream(task="Write a simple Hello world code ?"):  # type: ignore
    
    # Check if the returned object is the final result of the task
    if isinstance(message, TaskResult):
        # Print the reason why the task/conversation was stopped (e.g., max_turns reached, task complete)
        print("Stop Reason:", message.stop_reason)
    else:
        # Print the source agent's name and the message content
        print(message.source, message)


user id='670ade45-0192-43d9-ac95-8a7bf2b1e1f4' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 15, 19, 46, 10, 652592, tzinfo=datetime.timezone.utc) content='Write a simple Hello world code ?' type='TextMessage'
Complex_DSA_Solver id='153a9ccd-0d49-4a7a-9dbc-04c156323326' source='Complex_DSA_Solver' models_usage=RequestUsage(prompt_tokens=36, completion_tokens=41) metadata={} created_at=datetime.datetime(2025, 7, 15, 19, 46, 11, 512111, tzinfo=datetime.timezone.utc) content='Certainly! Here is a simple Python code to display "Hello, World!":\n\n```python\nprint("Hello, World!")\n```\n\nRunning this code will output "Hello, World!" to the console.' type='TextMessage'
CODE_REVEIWER id='53391252-aa13-44a9-9a08-bfb715b53569' source='CODE_REVEIWER' models_usage=RequestUsage(prompt_tokens=93, completion_tokens=8) metadata={} created_at=datetime.datetime(2025, 7, 15, 19, 46, 12, 769650, tzinfo=datetime.timezone.utc) content='Clear and efficient, no optimizati

In [6]:
# Importing the Console utility to display streamed messages interactively in the console
from autogen_agentchat.ui import Console

# Resetting the team to clear previous conversation history
await team.reset()  # Ensures a fresh run without leftover context

# Running the task and streaming the conversation to the console
# The Console() function wraps around the run_stream method and displays agent messages as they are generated
await Console(
    team.run_stream(task="Write a simple Hello world code.")  # Initial user prompt/task
)


---------- TextMessage (user) ----------
Write a simple Hello world code.
---------- TextMessage (Complex_DSA_Solver) ----------
Certainly! Here’s a simple "Hello, World!" program in Python:

```python
print("Hello, World!")
```

This code uses the `print` function to display the text "Hello, World!" to the screen.
---------- TextMessage (CODE_REVEIWER) ----------
The code is already optimized.
---------- TextMessage (CODE_EDITOR) ----------
Simple, clear, and efficient as is.
---------- TextMessage (Complex_DSA_Solver) ----------
Thank you! If you have any other requests or questions, feel free to ask.
---------- TextMessage (CODE_REVEIWER) ----------
No changes needed, code is optimal.
---------- TextMessage (CODE_EDITOR) ----------
The code is concise and effective.
---------- TextMessage (Complex_DSA_Solver) ----------
Great! If you have any other pieces of code you'd like reviewed or questions, feel free to reach out.
---------- TextMessage (CODE_REVEIWER) ----------
Code is alrea

TaskResult(messages=[TextMessage(id='dc521c65-ae4b-448a-aa5c-90e22824459d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 46, 23, 492209, tzinfo=datetime.timezone.utc), content='Write a simple Hello world code.', type='TextMessage'), TextMessage(id='c13af88b-77c7-44c5-b3c0-ad839e05be4c', source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=47), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 46, 24, 293617, tzinfo=datetime.timezone.utc), content='Certainly! Here’s a simple "Hello, World!" program in Python:\n\n```python\nprint("Hello, World!")\n```\n\nThis code uses the `print` function to display the text "Hello, World!" to the screen.', type='TextMessage'), TextMessage(id='2e928f19-5a2f-4e95-afa7-7d539fc51e52', source='CODE_REVEIWER', models_usage=RequestUsage(prompt_tokens=99, completion_tokens=6), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 46, 25, 540345, tzinfo=datetime.

In [7]:
await Console(team.run_stream(task="conitnue."))

---------- TextMessage (user) ----------
conitnue.
---------- TextMessage (Complex_DSA_Solver) ----------
If you have any specific requests or need help with another topic, feel free to let me know. I'm here to assist you!
---------- TextMessage (CODE_REVEIWER) ----------
Continue what exactly, please specify.
---------- TextMessage (CODE_EDITOR) ----------
Please clarify what you'd like to continue.
---------- TextMessage (Complex_DSA_Solver) ----------
If you can provide more details on what you need, I'd be happy to assist further!
---------- TextMessage (CODE_REVEIWER) ----------
Clarify continuation requirements, please.
---------- TextMessage (CODE_EDITOR) ----------
Please specify what you'd like to continue with.
---------- TextMessage (Complex_DSA_Solver) ----------
Let me know your requirements or what you'd like to work on next, and I'll be happy to assist!
---------- TextMessage (CODE_REVEIWER) ----------
Please provide continuation context.
---------- TextMessage (CODE_EDI

TaskResult(messages=[TextMessage(id='e6e4f9d2-46fb-42f6-a501-44849386d384', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 10, 765504, tzinfo=datetime.timezone.utc), content='conitnue.', type='TextMessage'), TextMessage(id='a51cf968-6f07-4d5e-9f7a-c2f166d31a98', source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=341, completion_tokens=26), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 11, 726683, tzinfo=datetime.timezone.utc), content="If you have any specific requests or need help with another topic, feel free to let me know. I'm here to assist you!", type='TextMessage'), TextMessage(id='96bc9eaf-f989-4020-a18d-b846590d273f', source='CODE_REVEIWER', models_usage=RequestUsage(prompt_tokens=385, completion_tokens=7), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 12, 792399, tzinfo=datetime.timezone.utc), content='Continue what exactly, please specify.', type='TextMessage'), TextMessage(

In [8]:
# Importing the AssistantAgent class to create agents with specific tasks
from autogen_agentchat.agents import AssistantAgent

# First agent: Starts from 0 and adds 1
add_1_agent_first = AssistantAgent(
    name='add_1_agent_first',  # Unique name for this agent
    model_client=model_client,  # Model used to generate the response (e.g., GPT-4o)
    system_message="Add 1 to the number, first number is 0. Give result as output"
    # Instruction: Start with 0, add 1, and return the result
)

# Second agent: Receives the output from the first agent and adds 1 again
add_1_agent_second = AssistantAgent(
    name='add_1_agent_second',  # Unique name
    model_client=model_client,  # Same model client
    system_message="Add 1 to the number you got from previous run. Give result as output."
    # Instruction: Add 1 to whatever result was produced before
)

# Third agent: Continues the process by adding 1 once more
add_1_agent_third = AssistantAgent(
    name='add_1_agent_third',
    model_client=model_client,
    system_message="Add 1 to the number from previous run. Give result as output."
    # Final step: Add 1 to the second agent's result
)


In [9]:
my_increment_team = RoundRobinGroupChat(participants=[add_1_agent_first,add_1_agent_second,add_1_agent_third],max_turns=3)

# Resume a Team

In [10]:
await Console(my_increment_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3


TaskResult(messages=[TextMessage(id='8cc189fd-f613-4a66-a6bd-d1a4231df9f3', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 28, 338980, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='2c47d993-0036-450e-b00f-8ed7132a8b11', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 28, 942227, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(id='6cc9b918-c80c-43bc-9f7a-e0e94dda26d0', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 29, 407138, tzinfo=datetime.timezone.utc), content='3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [11]:
await Console(my_increment_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
4
---------- TextMessage (add_1_agent_second) ----------
5
---------- TextMessage (add_1_agent_third) ----------
6


TaskResult(messages=[TextMessage(id='6869730f-e181-49a5-86a5-a3e5c335ff7d', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=50, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 31, 335057, tzinfo=datetime.timezone.utc), content='4', type='TextMessage'), TextMessage(id='8f58b8a9-4f03-45b3-9600-51e7fa270322', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=60, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 31, 887845, tzinfo=datetime.timezone.utc), content='5', type='TextMessage'), TextMessage(id='dcb3b483-8d15-493c-b6e4-c4f994a4ec70', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=67, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 32, 467172, tzinfo=datetime.timezone.utc), content='6', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [12]:
await team.reset()
await Console(my_increment_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
7
---------- TextMessage (add_1_agent_second) ----------
8
---------- TextMessage (add_1_agent_third) ----------
9


TaskResult(messages=[TextMessage(id='486d05bb-2960-4bb2-8081-b2d8603bc395', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=76, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 32, 916628, tzinfo=datetime.timezone.utc), content='7', type='TextMessage'), TextMessage(id='641ef435-098e-4426-869b-61931fb8b2c9', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=86, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 33, 400683, tzinfo=datetime.timezone.utc), content='8', type='TextMessage'), TextMessage(id='a3d2c45d-dda1-4c65-b300-4e7b4f500378', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=92, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 33, 880299, tzinfo=datetime.timezone.utc), content='9', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [13]:
await my_increment_team.reset()
await Console(my_increment_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3


TaskResult(messages=[TextMessage(id='ea80d6bd-cee6-41f1-8e67-c0be4d8cbed9', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 34, 759255, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='55ca58f1-08db-4281-a2b2-0490a61a3729', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 35, 303784, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(id='7cb827c3-973d-4d75-ae6a-5b172416bf74', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 15, 19, 47, 35, 817921, tzinfo=datetime.timezone.utc), content='3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')